#ANMNCPOP_Test_Data
# Causal Model Discovery Problems in Learning Joint Multiple Dynamical Systems via Non-Commutative Polynomial Optimization(NCPOP)

Welcome to the repository for generating ANMNCPOP test data! For further analysis, we can standardize real data or generate synthetic data following the below instruction.

# Synthetic_Data_Generation
* Weighted random DAG is produced according to num_nodes and num_edges. Test raw datasets is the time series generating from weighted random DAG and SEM type.

# Real_Data_Standardization
* For multiple features time series data, all time series will be saved as a (Feature_num, Sample_num, Time) three dimensions array for applying ANM-NCPOP.

# **Output Data Form**
Causality Data can be stored as NumPy array x(F features, S smples and T timesets) and matrix y(F features, F features) under a npz file.

#__Step 1: Get start__

* mount drive
* set envirment
* install packages

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir("/content/drive/MyDrive/Colab Notebooks/NCPOP-Colab Notebooks/Test_Causality_Datasets/Real_data/Krebs_Cycle/")
# os.chdir("/content/drive/MyDrive/Colab Notebooks/NCPOP-Colab Notebooks/Test_Causality_Datasets/Real_data/Krebs_Cycle/Details_Krebs_Cycle/MetricsDAG/")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#__Step 2: Generate Synthetic Data__
* Weighted random DAG is produced according to num_nodes and num_edges. Test raw datasets is the time series generating from weighted random DAG and SEM type.

In [ ]:
# from Generate_SyntheticData import*
from itertools import combinations
from BuiltinDataSet import DAG
from pickle import TRUE
import numpy as np
import pandas as pd
import networkx as nx
import logging
import tarfile
import os
import re

class Generate_SyntheticData(object):
    '''
    Simulate IID datasets for causal structure learning.

    Parameters
    ----------
    W: np.ndarray
        Weighted adjacency matrix for the target causal graph.
    n: int
        Number of samples for standard trainning dataset.
    T: int
        Number of timeseries for standard trainning dataset.
    method: str, (linear or nonlinear), default='linear'
        Distribution for standard trainning dataset.
    sem_type: str
        gauss, exp, gumbel, uniform, logistic (linear);
        mlp, mim, gp, gp-add, quadratic (nonlinear).
    noise_scale: float
        Scale parameter of noise distribution in linear SEM.
    '''

    def __init__(self, W, n=1000, T=20, method='linear',
                 sem_type='gauss', noise_scale=1.0):

        self.B = (W != 0).astype(int)
        if method == 'linear':
            self.XX = Generate_SyntheticData._simulate_linear_sem(
                    W, n, T, sem_type, noise_scale)
        elif method == 'nonlinear':
            self.XX = Generate_SyntheticData._simulate_nonlinear_sem(
                    W, n, T, sem_type, noise_scale)
        logging.info('Finished synthetic dataset')

    @staticmethod
    def _simulate_linear_sem(W, n, T, sem_type, noise_scale):
        """
        Simulate samples from linear SEM with specified type of noise.
        For uniform, noise z ~ uniform(-a, a), where a = noise_scale.

        Parameters
        ----------
        W: np.ndarray
            [d, d] weighted adj matrix of DAG.
        n: int
            Number of samples, n=inf mimics population risk.
        T: int
        Number of timeseries for standard trainning dataset.
        sem_type: str
            gauss, exp, gumbel, uniform, logistic.
        noise_scale: float
            Scale parameter of noise distribution in linear SEM.

        Return
        ------
        XX: np.ndarray
            [T, n, d] sample matrix, [d, d] if n and T=inf
        """
        def _simulate_single_equation(X, w, scale):
            """X: [n, num of parents], w: [num of parents], x: [n]"""
            if sem_type == 'gauss':
                z = np.random.normal(scale=scale, size=T)
                x = X @ w + z
            elif sem_type == 'exp':
                z = np.random.exponential(scale=scale, size=T)
                x = X @ w + z
            elif sem_type == 'gumbel':
                z = np.random.gumbel(scale=scale, size=T)
                x = X @ w + z
            elif sem_type == 'uniform':
                z = np.random.uniform(low=-scale, high=scale, size=T)
                x = X @ w + z
            elif sem_type == 'logistic':
                x = np.random.binomial(1, sigmoid(X @ w)) * 1.0
            else:
                raise ValueError('Unknown sem type. In a linear model, \
                                 the options are as follows: gauss, exp, \
                                 gumbel, uniform, logistic.')
            return x

        d = W.shape[0]
        if noise_scale is None:
            scale_vec = np.ones(d)
        elif np.isscalar(noise_scale):
            scale_vec = noise_scale * np.ones(d)
        else:
            if len(noise_scale) != d:
                raise ValueError('noise scale must be a scalar or has length d')
            scale_vec = noise_scale
        G_nx =  nx.from_numpy_array(W, create_using=nx.DiGraph)
        if not nx.is_directed_acyclic_graph(G_nx):
            raise ValueError('W must be a DAG')
        if np.isinf(T):  # population risk for linear gauss SEM
            if sem_type == 'gauss':
                # make 1/d X'X = true cov
                X = np.sqrt(d) * np.diag(scale_vec) @ np.linalg.inv(np.eye(d) - W)
                return X
            else:
                raise ValueError('population risk not available')
        # empirical risk
        ordered_vertices = list(nx.topological_sort(G_nx))
        assert len(ordered_vertices) == d
        X = np.zeros([T, d])
        XX = np.zeros((d, n, T))
        for j in ordered_vertices:
            parents = list(G_nx.predecessors(j))
            X[:, j] = _simulate_single_equation(X[:, parents], W[parents, j], scale_vec[j])
        for ns in range(n):
            XX[:, ns] = np.transpose(X)
        return XX

    @staticmethod
    def _simulate_nonlinear_sem(W, n, T, sem_type, noise_scale):
        """
        Simulate samples from nonlinear SEM.

        Parameters
        ----------
        B: np.ndarray
            [d, d] binary adj matrix of DAG.
        n: int
            Number of samples.
        T: int
            Number of times.
        sem_type: str
            mlp, mim, gp, gp-add, or quadratic.
        noise_scale: float
            Scale parameter of noise distribution in linear SEM.

        Return
        ------
        XX: np.ndarray
            [T, n, d] sample matrix
        """
        if sem_type == 'quadratic':
            return GenerateData._simulate_quad_sem(W, T, noise_scale)

        def _simulate_single_equation(X, scale):
            """X: [n, num of parents], x: [n]"""
            z = np.random.normal(scale=scale, size=n)
            pa_size = X.shape[1]
            if pa_size == 0:
                return z
            if sem_type == 'mlp':
                hidden = 100
                W1 = np.random.uniform(low=0.5, high=2.0, size=[pa_size, hidden])
                W1[np.random.rand(*W1.shape) < 0.5] *= -1
                W2 = np.random.uniform(low=0.5, high=2.0, size=hidden)
                W2[np.random.rand(hidden) < 0.5] *= -1
                x = sigmoid(X @ W1) @ W2 + z
            elif sem_type == 'mim':
                w1 = np.random.uniform(low=0.5, high=2.0, size=pa_size)
                w1[np.random.rand(pa_size) < 0.5] *= -1
                w2 = np.random.uniform(low=0.5, high=2.0, size=pa_size)
                w2[np.random.rand(pa_size) < 0.5] *= -1
                w3 = np.random.uniform(low=0.5, high=2.0, size=pa_size)
                w3[np.random.rand(pa_size) < 0.5] *= -1
                x = np.tanh(X @ w1) + np.cos(X @ w2) + np.sin(X @ w3) + z
            elif sem_type == 'gp':
                from sklearn.gaussian_process import GaussianProcessRegressor
                gp = GaussianProcessRegressor()
                x = gp.sample_y(X, random_state=None).flatten() + z
            elif sem_type == 'gp-add':
                from sklearn.gaussian_process import GaussianProcessRegressor
                gp = GaussianProcessRegressor()
                x = sum([gp.sample_y(X[:, i, None], random_state=None).flatten()
                        for i in range(X.shape[1])]) + z
            else:
                raise ValueError('Unknown sem type. In a nonlinear model, \
                                 the options are as follows: mlp, mim, \
                                 gp, gp-add, or quadratic.')
            return x

        B = (W != 0).astype(int)
        d = B.shape[0]
        if noise_scale is None:
            scale_vec = np.ones(d)
        elif np.isscalar(noise_scale):
            scale_vec = noise_scale * np.ones(d)
        else:
            if len(noise_scale) != d:
                raise ValueError('noise scale must be a scalar or has length d')
            scale_vec = noise_scale
        X = np.zeros([T, d])
        XX = np.zeros((T, n, d))
        G_nx =  nx.from_numpy_array(B, create_using=nx.DiGraph)
        ordered_vertices = list(nx.topological_sort(G_nx))
        assert len(ordered_vertices) == d
        for j in ordered_vertices:
            parents = list(G_nx.predecessors(j))
            X[:, j] = _simulate_single_equation(X[:, parents], scale_vec[j])
        for ns in range(n):
            XX[:, ns] = X

        return XX

In our example, Weighted random DAG and raw datasets are saved under LinearGauss_6_15_TS.npz as a casaul matrix x and an array y, separately.


In [ ]:
if __name__ == "__main__":
    ############################################################################################################
    ############################################ SETTING File_PATH and file_name ###############################
    ############################################################################################################

    File_PATH = "/content/drive/MyDrive/Colab Notebooks/NCPOP-Colab Notebooks/Test_Causality_Datasets/Real_data/Krebs_Cycle/"
    method = 'linear'
    sem_type = 'gauss'
    num_nodes = 6
    num_edges = 15
    num_datasets = 10
    T=20
    # Weighted adjacency matrix for the target causal graph
    weighted_random_dag = DAG.erdos_renyi(n_nodes=num_nodes, n_edges=num_edges, seed=1)
    # _simulate_linear_sem(W =weighted_random_dag, n = num_datasets, sem_type = 'gauss', noise_scale=1.0)
    dataset = Generate_SyntheticData(W=weighted_random_dag, n=num_datasets, T=20, method=method, sem_type=sem_type)
    true_dag, data = dataset.B, dataset.XX
    sname = method.capitalize()+sem_type.capitalize()+'_'+str(num_nodes)+'_'+str(num_edges)+'_TS.npz'
    np.savez(File_PATH + sname, x=dataset.XX, y=dataset.B)
    print('INFO: Check for '+sname+ '!')

INFO: Check for LinearGauss_6_15_TS.npz!


#__Step 3: Real Data Standardization__
* The data can be any format that is supported by the Real_Data_Standardization() function, currently including npy, tar.gz, csv and tsv.




In [ ]:
# from Data_Standardization import*
from pickle import TRUE
import os
import re
import numpy as np
import pandas as pd
import tarfile
from itertools import combinations

class Real_Data_Standardization(object):
    '''
    A class for preparing data to simulate random (causal) DAG.

    Parameters
    ----------
    File_PATH: str
        Save file path
    File_NAME: str
        Read data name
    '''

    def __init__(self, File_PATH, Data_NAME):
        self.File_PATH = File_PATH
        self.Data_NAME = Data_NAME

    def Produce_Rawdata(self):

        def readable_File(FilePATH):
            read_Dir=os.listdir(FilePATH)
            count = 0
            readable_F = []
            for f in read_Dir:
                file = os.path.join(FilePATH, f)
                if os.path.isdir(file):
                    count = count+1
                else:
                    readable_F.append(f)
            return count,readable_F

        self.Read_File = readable_File(self.File_PATH)[1]
        self.TS_path = self.File_PATH + self.Data_NAME + '_TS/'

        # Check empty files under riute
        if len(self.File_PATH) == 0:
            print('INFO: No Data Under the Current Route!')
        else:
            File_NAME = []
            File_TYPE = []

            # Delete files and list readable Files
            for i in self.Read_File:
                File_NAME.append(re.split("\.", i)[0])
                File_TYPE.append(re.split("\.", i)[1])

            ###################################### Deal with Two Dimensions Causality Data ###################################
            if self.Data_NAME+'.npz' in self.Read_File:
                Tests_data = np.load(self.File_PATH + self.Data_NAME+'.npz', allow_pickle=True)
                Raw_data = Tests_data['x']
                true_dag = Tests_data['y']
                print('INFO: Check for '+self.Data_NAME +'.npz'+ '!')

            elif self.Data_NAME+'.tar.gz' in self.Read_File:
                # open file
                file = tarfile.open(self.File_PATH + self.Data_NAME + '.tar.gz')

                # print file names
                file_names = file.getnames()
                print(file_names)

                # extract files
                file.extractall(self.File_PATH)

                # close file
                file.close()

                Raw_data = pd.read_csv(self.File_PATH+file_names[1])
                true_dag = np.load(self.File_PATH+file_names[2])

                # save numpy to npz file
                np.savez(self.File_PATH + self.Data_NAME+'.npz', x=Raw_data , y=true_dag)
                print('INFO: Check for '+self.Data_NAME +'.npz'+ '!')

            elif self.Data_NAME+'.csv' in self.Read_File:
                Raw_data = pd.read_csv(self.File_PATH+ self.Data_NAME+'.csv', header=0, index_col=0)
                true_dag = pd.read_csv(self.File_PATH+'true_graph.csv', header=0, index_col=0)

                # save numpy to npz file
                np.savez(self.File_PATH + self.Data_NAME+'.npz', x=Raw_data , y=true_dag)
                print('INFO: Check for '+self.Data_NAME +'.npz'+ '!')



            ################################ Deal with Multi-dimensions Causality Data ###################################
            elif os.path.exists(self.TS_path):
                read_Dir_TS=os.listdir(self.TS_path)
                # true_dag = pd.read_csv(self.File_PATH+'true_graph.csv', header=0, index_col=0)
                '''
                Timeseries_List_path = self.File_PATH+'series_list.csv'
                Read_Timeseries = pd.read_csv(Timeseries_List_path)
                print(len(Read_Timeseries), len(read_Dir_TS))
                if len(Read_Timeseries) >= len(read_Dir_TS):
                    print('INFO: Start Analyzing '+ self.Data_NAME + ' Time Series File!')
                    TS_List = read_Dir_TS
                else:
                    print('INFO: Start Analyzing '+ self.Data_NAME + ' Time Series List!')
                    TS_List = Read_Timeseries['Series_num']
                '''
                true_graph = np.load(self.File_PATH+'true_graph.npz')

                labels = ["FUMARATE", "GTP", "H2O", "CIS-ACONITATE", "MALATE",
                "OXALOACETATE", "FAD", "SUCCINYL-COA", "NAD",
                          "A-K-GLUTARATE", "GDP", "NADH", "CITRATE", "SUCCINATE",
                "ISOCITRATE", "ACETY-COA"]
                true_dag = pd.DataFrame(true_graph['arr_0'],  index=labels, columns=labels)
                # print(true_dag)
                lds = pd.read_csv(self.TS_path+ read_Dir_TS[0], delimiter='\t', index_col=0, header=None)
                feature_name = np.array(lds.index)
                # lds_trans = np.transpose(lds)
                # feature_name = lds_trans.columns
                feature_num = len(feature_name)
                sample_num = len(read_Dir_TS)
                T_num = lds.shape[1]
                # if labels == feature_name:
                Raw_data = np.zeros((feature_num, sample_num, T_num))
                for ns in range(sample_num):
                    X = pd.read_csv(self.TS_path+ read_Dir_TS[ns], delimiter='\t', index_col=0, header=None)
                    X_trans = np.transpose(X)
                    for fn in range(feature_num):
                        Raw_data[fn, ns, :] = list(X_trans[feature_name[fn]])
                    '''
                    matrix = np.zeros((d, d))
                    np.fill_diagonal(matrix, 0)
                    np.fill_diagonal(matrix[:, 1:], 1)
                    np.savez(self.File_PATH + self.Data_NAME+'.npz', x=Raw_data , y=matrix)
                    print('INFO: Check for '+self.Data_NAME +'.npz'+ '!')

                    '''
                non_zero_count = np.count_nonzero(true_dag)

                # save numpy to npz file
                sname = self.Data_NAME + '_'+str(feature_num)+'_'+str(non_zero_count)+'_TS.npz'
                np.savez(self.File_PATH + sname, x=Raw_data , y=true_dag)
                print('INFO: Check for '+sname + '!')

            else:
                print('INFO: Wrong DataType!')


In our example, multiple features time series are saved under Krebs_Cycle.npz as a casaul matrix and an array, separately.

* __x__: is an array in shape(F, S, T), where the number of row F is features_num, the number of column S is smples_num and the number of deep T is timesets.
* __y__: is a nonsymmetric square matrix.

In [ ]:
if __name__ == "__main__":
    ############################################################################################################
    ############################################ SETTING File_PATH and file_name ###############################
    ############################################################################################################

    File_PATH = "/content/drive/MyDrive/Colab Notebooks/NCPOP-Colab Notebooks/Test_Causality_Datasets/Real_data/Krebs_Cycle/"
    file_name = 'Krebs_Cycle'
    dt = Real_Data_Standardization(File_PATH, file_name)
    dt.Produce_Rawdata()

INFO: Check for Krebs_Cycle_16_43_TS.npz!


#__Step 4: Test Data__
* Output Data Form
Causality Data including NumPy array x and matrix y are stored under a npz file, which is rawdata generating from true_dag or observation data in real case as well
as a ground true casaul matrix.

* **Raw_data:**

An array saves F features, S smples and T timesets Time Series.

* **True_dag:**

A causal matrix is a nonsymmetric matrix saved as the shape of (F features, F features)

In our examples, Raw_data is saved under npz file, where the first number is the number of row, second number is the number of column and third number is the number of deep.



In [ ]:
if __name__ == "__main__":
    ############################################################################################################
    ############################################ SETTING File_PATH and file_name ###############################
    ############################################################################################################

    File_PATH = "/content/drive/MyDrive/Colab Notebooks/NCPOP-Colab Notebooks/Test_Causality_Datasets/Real_data/Krebs_Cycle/"
    file_name = 'Krebs_Cycle_16_43_TS'
    data = np.load(File_PATH+file_name+'.npz')
    Raw_data = data['x']
    true_dag = data['y']
    print(Raw_data.shape, true_dag.shape)

(16, 120, 500) (16, 16)


In [ ]:
if __name__ == "__main__":
    ############################################################################################################
    ############################################ SETTING File_PATH and file_name ###############################
    ############################################################################################################

    File_PATH = "/content/drive/MyDrive/Colab Notebooks/NCPOP-Colab Notebooks/Test_Causality_Datasets/Real_data/Krebs_Cycle/"
    file_name = 'LinearGauss_6_15_TS'
    data = np.load(File_PATH+file_name+'.npz')
    Raw_data = data['x']
    true_dag = data['y']
    print(Raw_data.shape, true_dag.shape)

(6, 10, 20) (6, 6)


#__Backup__

In [ ]:
import pandas as pd
X = pd.read_csv('Krebs_Cycle_TS/threes37.tsv', delimiter='\t', index_col=0, header=None)
X.index
print(np.array(X.index))
df = np.transpose(X)
# list(df[df.columns[1]])

['FUMARATE' 'GTP' 'H2O' 'CIS-ACONITATE' 'MALATE' 'OXALOACETATE' 'FAD'
 'SUCCINYL-COA' 'NAD' 'A-K-GLUTARATE' 'GDP' 'NADH' 'CITRATE' 'SUCCINATE'
 'ISOCITRATE' 'ACETY-COA']


In [ ]:
import pandas as pd
data = np.load('true_graph.npz')

labels = ["FUMARATE", "GTP", "H2O", "CIS-ACONITATE", "MALATE",
"OXALOACETATE", "FAD", "SUCCINYL-COA", "NAD",
           "A-K-GLUTARATE", "GDP", "NADH", "CITRATE", "SUCCINATE",
"ISOCITRATE", "ACETY-COA"]
true_dag = pd.DataFrame(data['arr_0'],  index=labels, columns=labels)
print(true_dag)
# list(df[df.columns[1]])

In [ ]:
class NCPOLR(object):
    """Estimator based on NCPOP Regressor

    References
    ----------
    Quan Zhou https://github.com/Quan-Zhou/Proper-Learning-of-LDS/blob/master/ncpop/functions.py

    Examples
    --------
    """

    def __init__(self, **kwargs):
        super(NCPOLR, self).__init__()


    def generate_multioperators(self, name, n_vars, m_vars, hermitian=None, commutative=False):
        """Generates a number of commutative or noncommutative operators

        :param name: The prefix in the symbolic representation of the noncommuting
                     variables. This will be suffixed by a number from 0 to
                     n_vars-1 if n_vars > 1.
        :type name: str.
        :param n_vars: The number of variables.
        :type n_vars: int.
        :param hermitian: Optional parameter to request Hermitian variables .
        :type hermitian: bool.
        :param commutative: Optional parameter to request commutative variables.
                            Commutative variables are Hermitian by default.
        :type commutative: bool.

        :returns: list of :class:`sympy.physics.quantum.operator.Operator` or
                  :class:`sympy.physics.quantum.operator.HermitianOperator`
                  variables

        :Example:

        >>> generate_variables('y', 2, commutative=True)
        #[y0, y1]
        """

        variables = []
        variables1 = []
        variables2 = []
        for i in range(n_vars):
            if n_vars > 1:
                var_name1 = '%s%s' % (name, i)
            else:
                var_name1 = '%s' % name
            if hermitian is not None and hermitian:
                variables1.append(HermitianOperator(var_name1))
            else:
                variables1.append(Operator(var_name1))
            variables1[-1].is_commutative = commutative
        for n in range(len(variables1)):
            for j in range(m_vars):
                if m_vars > 1:
                    var_name = '%s%s' % (variables1[n], j)
                else:
                    var_name = '%s' % variables1[n]
                if hermitian is not None and hermitian:
                    variables2.append(HermitianOperator(var_name))
                else:
                    variables2.append(Operator(var_name))
                variables2[-1].is_commutative = commutative
        var = np.matrix(np.array(variables2).reshape(n_vars,m_vars))
        var = np.array(variables2).reshape(n_vars,m_vars)
        #print(variables1,variables2)
        return var


    def estimate(self, Y, m,level):
        """Fit Estimator based on NCPOP Regressor model and predict y or produce residuals.
        The module converts a noncommutative optimization problem provided in SymPy
        format to an SDPA semidefinite programming problem.
        Define a function for solving the NCPO problems with
        given standard deviations of process noise and observtion noise,
        length of  estimation data and required relaxation level.

        Parameters
        ----------
        Y: array
            Variable seen as input
        m:
        n:

        Returns
        -------
        y_predict: array
            regression predict values of y or residuals
        obj: num
            Objective value in optima
        """
        n=7
        T= len(Y)
        Y = np.transpose(Y)
        # Decision Variables
        G = NCPOLR().generate_multioperators("G", n_vars=n, m_vars=n, hermitian=True, commutative=False)
        Fdash = NCPOLR().generate_multioperators("Fdash", n_vars=m, m_vars=n, hermitian=True, commutative=False)
        phi = NCPOLR().generate_multioperators("phi", n_vars=n, m_vars=T+1, hermitian=True, commutative=False)
        q = NCPOLR().generate_multioperators("q", n_vars=n, m_vars=T, hermitian=True, commutative=False)
        p = NCPOLR().generate_multioperators("p", n_vars=m, m_vars=T, hermitian=True, commutative=False)
        f = NCPOLR().generate_multioperators("f", n_vars=m, m_vars=T, hermitian=True, commutative=False)


        # Objective
        #obj = sum((Y[i]-f[i])**2 for i in range(T)) + 0.0005*sum(p[i]**2 for i in range(T)) + 0.0001*sum(q[i]**2 for i in range(T))
        obj = sum((Y[mm][i])*2 for i in range(T) for mm in range(m))+ 0.0005*sum(p[mm][i]*2 for i in range(T) for mm in range(m)) + 0.0001*sum(q[nn][i]*2 for i in range(T) for nn in range(n))

        # Constraints

        ine1 = [f[mm][i] - Fdash[mm][nn]*phi[nn][i+1] - p[mm][i] for nn in range(n) for i in range(T) for mm in range(m)]
        ine2 = [-f[mm][i] + Fdash[mm][nn]*phi[nn][i+1] + p[mm][i] for nn in range(n) for i in range(T) for mm in range(m)]
        ine3 = [phi[nn][i+1] - G[nn][nn]*phi[nn][i] - q[nn][i] for nn in range(n) for i in range(T)]
        ine4 = [-phi[nn][i+1] + G[nn][nn]*phi[nn][i] + q[nn][i] for i in range(T) for nn in range(n)]
        #ine5 = [(Y[i]-f[i])**2 for i in range(T)]
        ines = ine1+ine2+ine3+ine4

        # Solve the NCPO
        GG = Operator(np.asarray(G).reshape(-1))
        FFdash = Operator(np.asarray(Fdash).reshape(-1))
        f = Operator(np.asarray(f).reshape(-1))
        q = Operator(np.asarray(q).reshape(-1))
        p = Operator(np.asarray(p).reshape(-1))
        phi = Operator(np.asarray(phi).reshape(-1))
        #print([Operator(GG),Operator(FFdash),Operator(f),Operator(p),Operator(phi),Operator(q)])

        sdp = SdpRelaxation(variables = flatten([GG,FFdash,f,p,phi,q]),verbose = 1)
        sdp.get_relaxation(level, objective=obj, inequalities=ines)
        sdp.solve(solver='mosek')
        '''
        Attributes:
          - `monomial_sets`: The monomial sets that generate the moment matrix blocks.

          - `monomial_index`: Dictionary that maps monomials to SDP variables.

          - `constraints`: The complete set of constraints after preprocesssing.

          - `primal`: The primal optimal value.

          - `dual`: The dual optimal value.

          - `x_mat`: The primal solution matrix.

          - `y_mat`: The dual solution matrix.

          - `solution_time`: The amount of time taken to solve the relaxation.

          - `status`: The solution status of the relaxation.
        '''
        sdp.write_to_file("slutions.csv")
        sdp.write_to_file('example.dat-s')
        sdp.find_solution_ranks()
        #sdp.solve(solver='sdpa', solverparameters={"executable":"sdpa_gmp","executable": "C:/Users/zhouq/Documents/sdpa7-windows/sdpa.exe"})
        print(sdp.primal, sdp.dual,sdp.status)
        return sdp.primal
        '''
        if(sdp.status != 'infeasible'):
            print('ok.')
            f_pred = []
            p_pred = []
            phi_pred = []
            q_pred = []
            for t in range(len(np.asarray(p).reshape(-1))):
                f_pred.append(sdp[f[t]])
                p_pred.append(sdp[p[t]])
            for t in range(len(np.asarray(q).reshape(-1))):
                phi_pred.append(sdp[phi[t]])
                q_pred.append(sdp[q[t]])
            Para = [sdp[GG].reshape(n,n),sdp[FFdash].reshape(m,n)]
            return f_pred.reshape(m,T),p_pred.reshape(m,T),phi_pred.reshape(n,T),p_pred.reshape(n,T),Para
            '''


In [ ]:
from itertools import product
X=HBP_Data
T= len(X)
#Y = np.transpose(test_tsdata)
m= len(np.transpose(X))
print(T,m)
K = 2
#L= np.array(np.random.randint(0,K,m))
L = [1,1,0,0,0]
level = 1
X_Group0 = X.iloc[:,np.flatnonzero(L)]
X_Group0 = np.array(X_Group0)#.reshape(-1)
N0 = len(np.transpose(X_Group0))

X_Group1 = X.iloc[:,np.flatnonzero(list(abs(np.asarray(L) -1)))]
X_Group1 = np.array(X_Group1)#.reshape(-1)
N1 = len(np.transpose(X_Group1))
print(N0,N1)
primal_value = NCPOLR().estimate(X_Group0,N0,level)+ NCPOLR().estimate(X_Group1,N1,level)

#y_LDS2_predict = NCPOLR().estimate(list(y_LDS2),level)
print(print(L,primal_value))

# itaration
l = [0,1]
prod = list(product(l, repeat=m))

for ll in range(len(prod)):
    L_new= np.array(prod[ll])
    if len([i for i in L_new if i==1]) >1 and len([i for i in L_new if i==0]) >1:
        #Separate into K=2 groups
        X_Group00 = X.iloc[:,np.flatnonzero(L_new)]
        X_Group00 = np.array(X_Group00)#.reshape(-1)
        N00 = len(np.transpose(X_Group00))

        X_Group11 = X.iloc[:,np.flatnonzero(list(abs(np.asarray(L_new) -1)))]
        X_Group11 = np.array(X_Group11)#.reshape(-1)
        N11 = len(np.transpose(X_Group11))
        print(X_Group00,X_Group11)

        primal_value_new = NCPOLR().estimate(X_Group00,N00,level)+ NCPOLR().estimate(X_Group11,N11,level)
        print(L_new,primal_value_new)
        if primal_value_new < primal_value:
            L= L_new
            primal_value = primal_value_new
            print(L,primal_value)
        else:
            continue
    else:
        continue
#return L




20 5
2 3
The problem has 6 noncommuting Hermitian variables
Calculating block structure...
Estimated number of SDP variables: 90
Generating moment matrix...
Reduced number of SDP variables: 90 90 (done: 101.11%, ETA 00:00:-0.0)
Processing 137/840 constraints...

Processing 840/840 constraints...
Problem
  Name                   :                 
  Objective sense        : minimize        
  Type                   : CONIC (conic optimization problem)
  Constraints            : 90              
  Affine conic cons.     : 0               
  Disjunctive cons.      : 0               
  Cones                  : 0               
  Scalar variables       : 0               
  Matrix variables       : 1 (scalarized: 364231)
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - primal attempts        : 1                 successes              : 1               
Lin. dep. 

Processing 1120/1120 constraints...
Problem
  Name                   :                 
  Objective sense        : minimize        
  Type                   : CONIC (conic optimization problem)
  Constraints            : 90              
  Affine conic cons.     : 0               
  Disjunctive cons.      : 0               
  Cones                  : 0               
  Scalar variables       : 0               
  Matrix variables       : 1 (scalarized: 642411)
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - primal attempts        : 1                 successes              : 1               
Lin. dep

Processing 840/840 constraints...
Problem
  Name                   :                 
  Objective sense        : minimize        
  Type                   : CONIC (conic optimization problem)
  Constraints            : 90              
  Affine conic cons.     : 0               
  Disjunctive cons.      : 0               
  Cones                  : 0               
  Scalar variables       : 0               
  Matrix variables       : 1 (scalarized: 364231)
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - primal attempts        : 1                 successes              : 1               
Lin. dep. 

Processing 1120/1120 constraints...
Problem
  Name                   :                 
  Objective sense        : minimize        
  Type                   : CONIC (conic optimization problem)
  Constraints            : 90              
  Affine conic cons.     : 0               
  Disjunctive cons.      : 0               
  Cones                  : 0               
  Scalar variables       : 0               
  Matrix variables       : 1 (scalarized: 642411)
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - primal attempts        : 1                 successes              : 1               
Lin. dep

...

Processing 840/840 constraints...
Problem
  Name                   :                 
  Objective sense        : minimize        
  Type                   : CONIC (conic optimization problem)
  Constraints            : 90              
  Affine conic cons.     : 0               
  Disjunctive cons.      : 0               
  Cones                  : 0               
  Scalar variables       : 0               
  Matrix variables       : 1 (scalarized: 364231)
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - primal attempts        : 1                 successes              : 1               
Lin. dep. 

Processing 1120/1120 constraints...
Problem
  Name                   :                 
  Objective sense        : minimize        
  Type                   : CONIC (conic optimization problem)
  Constraints            : 90              
  Affine conic cons.     : 0               
  Disjunctive cons.      : 0               
  Cones                  : 0               
  Scalar variables       : 0               
  Matrix variables       : 1 (scalarized: 642411)
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - primal attempts        : 1                 successes              : 1               
Lin. dep

Processing 840/840 constraints...
Problem
  Name                   :                 
  Objective sense        : minimize        
  Type                   : CONIC (conic optimization problem)
  Constraints            : 90              
  Affine conic cons.     : 0               
  Disjunctive cons.      : 0               
  Cones                  : 0               
  Scalar variables       : 0               
  Matrix variables       : 1 (scalarized: 364231)
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - primal attempts        : 1                 successes              : 1               
Lin. dep. 

Processing 1120/1120 constraints...
Problem
  Name                   :                 
  Objective sense        : minimize        
  Type                   : CONIC (conic optimization problem)
  Constraints            : 90              
  Affine conic cons.     : 0               
  Disjunctive cons.      : 0               
  Cones                  : 0               
  Scalar variables       : 0               
  Matrix variables       : 1 (scalarized: 642411)
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - primal attempts        : 1                 successes              : 1               
Lin. dep

Processing 840/840 constraints...
Problem
  Name                   :                 
  Objective sense        : minimize        
  Type                   : CONIC (conic optimization problem)
  Constraints            : 90              
  Affine conic cons.     : 0               
  Disjunctive cons.      : 0               
  Cones                  : 0               
  Scalar variables       : 0               
  Matrix variables       : 1 (scalarized: 364231)
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - primal attempts        : 1                 successes              : 1               
Lin. dep. 